# mXGWR implementation
faster implementation than python. note: SEM model and GWR conducted in python gwr.ipynb notebook

In [ ]:
.libPaths("~/R/x86_64-pc-linux-gnu-library/4.4")

install.packages("sf", type = "source", verbose = TRUE, keep_outputs = TRUE)
install.packages("Rcpp")
find.package("Rcpp")
find.package("RcppEigen")
library(Rcpp)
library(RcppEigen)
library(sf)
library(spatstat)
library(tidyverse)
library(GWmodel)    # to undertake the GWR
library(geojsonio)  # to read the geojson file
library(spatialRF)
#dplyr
library(dplyr)

.libPaths()
install.packages("geojsonio",verbose = TRUE, keep_outputs = TRUE)
install.packages("GWmodel",type = "source", verbose = TRUE, keep_outputs = TRUE)
install.packages("spatstat",type = "source", verbose = TRUE, keep_outputs = TRUE)
install.packages("spatialRF")
install.packages("kableExtra")


https://bookdown.org/lexcomber/GEOG3195/spatial-models---geographically-weighted-regression.html

In [2]:
xenium_aligned_ductal_invasive = read.csv("results/xenium_aligned_ductal_invasive.csv",row.names=1)
#print unique xenium_aligned_ductal_invasive Cluster values
print(unique(xenium_aligned_ductal_invasive$Cluster))
X_pca_df_pcs=read.csv("results/X_pca_df.csv",row.names=1)
#select columns starting with PC and cell_id
X_pca_df_pcs = X_pca_df_pcs %>% select(starts_with("PC"),cell_id)


[1] "DCIS_2"                "Invasive_Tumor"        "Prolif_Invasive_Tumor"
[4] "DCIS_1"               


In [ ]:
#remove cluster column
X_pca_df = X_pca_df %>% select(-Cluster)
#merge X_pca_df with X_pca_df_pcs
X_pca_df = merge(X_pca_df,X_pca_df_pcs,by='cell_id')
xenium_aligned_ductal_invasive = read.csv("results/xenium_aligned_ductal_invasive.csv",row.names=1)
#filter for Cluster and Barcode
xenium_aligned_ductal_invasive = xenium_aligned_ductal_invasive %>% select(Cluster,Barcode)

#merge the two dataframes
xenium_aligned_ductal_invasive_merged = merge(xenium_aligned_ductal_invasive,X_pca_df,by.y='cell_id',by.x="Barcode")
#save xenium_aligned_ductal_invasive_merged
write.csv(xenium_aligned_ductal_invasive_merged,"results/xenium_regression_df.csv")
subtype_specific_model="Invasive_Tumor"
#subtype_specific_model="alldcis"

##filter for the specific subtype
#xenium_aligned_ductal_invasive_merged = xenium_aligned_ductal_invasive_merged %>% filter(Cluster==subtype_specific_model)
#filter for if Cluster contains DCIS
xenium_aligned_ductal_invasive_merged = xenium_aligned_ductal_invasive_merged %>% filter(grepl("DCIS",Cluster))
#drop the cluster column
X_pca_df = xenium_aligned_ductal_invasive_merged %>% select(-Cluster)
X_pca_df = xenium_aligned_ductal_invasive_merged
#remove the barcode column
X_pca_df = X_pca_df %>% select(-Barcode)


#NORMALIZE THE COLUMNS BEGINNING WITH "MIN"
X_pca_df = X_pca_df %>% mutate_at(vars(starts_with("min")), funs((.-min(.))/(max(.)-min(.))))
#switch the sign of the columns beginning with "min"
X_pca_df = X_pca_df %>% mutate_at(vars(starts_with("min")), funs(-.))

X_pca_df = X_pca_df %>% mutate_at(vars(starts_with("PC")), funs((.-min(.))/(max(.)-min(.))))
sf_data <- st_as_sf(
  X_pca_df,
  coords = c("array_col", "array_row"), # Specify coordinate columns
  crs = NA                              # No CRS
)

#select predictor.variable.names as columns starting with "PC" or "min"
predictor.variable.names = colnames(X_pca_df)[grepl("PC",colnames(X_pca_df)) | grepl("min_distance",colnames(X_pca_df))]


Code to run to remove variables with high VIF

In [ ]:
"""
install.packages("corrplot")
library(corrplot)
library(RColorBrewer)
options(repr.plot.width = 12, repr.plot.height = 10)  # Adjust width and height as needed
correlation_matrix = cor(X_pca_df[,predictor.variable.names])

corrplot(correlation_matrix, type="upper", order="hclust",
         col=brewer.pal(n=8, name="RdBu"))

predictor.variable.names <- spatialRF::auto_cor(
  x = X_pca_df[, predictor.variable.names],
  cor.threshold = 0.90) %>% 
  spatialRF::auto_vif(
    vif.threshold = 10,preference.order = preference)
  
  predictor.variable.names <- spatialRF::auto_cor(x = X_pca_df[, predictor.variable.names],cor.threshold = 0.80)
"""

High VIF variables

In [ ]:
X_pca_df = X_pca_df %>% select(-min_distance_to_Endothelial.RGS5,-min_distance_to_CAFs.MSC.iCAF.like)

Change filtering options

In [ ]:
#select column names starting with "min" and "PC" from X_pca_df and "array_col","array_row"
#predictor.variable.names=colnames(X_pca_df)[grepl("min_",colnames(X_pca_df)) | grepl("PC",colnames(X_pca_df))]
predictor.variable.names=colnames(X_pca_df)[grepl("min_",colnames(X_pca_df))]

dependent_variable <- "EMT_hallmarks"
formula_str <- paste(dependent_variable, "~", paste(predictor.variable.names, collapse = " + "))
formula <- as.formula(formula_str)
sf_data <- st_as_sf(
  X_pca_df,
  coords = c("array_col", "array_row"), 
  crs = NA                           
)


Detect bandwidths for each variable

In [ ]:

bw <- bw.gwr(formula,approach = "AIC",
             adaptive = T,
             data=sf_data) 

m.gwr <- gwr.basic(formula, 
                adaptive = T,
                data = sf_data,
                bw = bw)  
#save the model
#save to subtype_specific_model + "_gwr.rds"
saveRDS(m.gwr,"justtme_dcis_gwr.rds")

In [ ]:
m.gwr

   ***********************************************************************
   *                       Package   GWmodel                             *
   ***********************************************************************
   Program starts at: 2024-11-02 00:27:08.857003 
   Call:
   gwr.basic(formula = formula, data = sf_data, bw = bw, adaptive = T)

   Dependent (y) variable:  EMT_hallmarks
   Independent variables:  min_distance_to_Cancer.LumA.SC min_distance_to_Macrophage min_distance_to_Cancer.Basal.SC min_distance_to_Cancer.LumB.SC min_distance_to_Luminal.Progenitors min_distance_to_PVL.Immature min_distance_to_Mature.Luminal min_distance_to_Cancer.Her2.SC min_distance_to_Myoepithelial min_distance_to_CAFs.myCAF.like min_distance_to_Cancer.Cycling min_distance_to_Monocyte min_distance_to_Cycling_Myeloid min_distance_to_Endothelial.Lymphatic.LYVE1 min_distance_to_Endothelial.CXCL12 min_distance_to_NK.cells min_distance_to_Cycling.T.cells min_distance_to_Endothelial.ACKR1 min_dis

In [ ]:
#list of bws0 based on the number of variables plus 1
bws0_list = rep(100,length(predictor.variable.names)+1)
gw.ms <- gwr.multiscale(formula, 
                        data = sf_data,
                        adaptive = T, max.iterations = 1000,
                        criterion="CVR",
                        kernel = "bisquare",
                        bws0=bws0_list,
                        verbose = T, predictor.centered=rep(T, 4))
saveRDS(gw.ms,"justtme_dcis_msgwr.rds")

Warning message in gwr.multiscale(formula, data = sf_data, adaptive = T, max.iterations = 1000, :
“All the predictors will be centered, please check the parameter predictor.centered”


In [3]:
m.gwr= readRDS("results/justtme_dcis_gwr.rds")
gw.ms=readRDS("results/justtme_dcis_msgwr.rds")


Extract coefficients to plot

In [ ]:

# Extract the coefficient and standard error columns for 'min_distance_to_CAFs.myCAF.like'
coefficients <- gw.ms$SDF$min_distance_to_T.cells.CD8.  # Coefficients
t_values <- gw.ms$SDF$min_distance_to_T.cells.CD8._TV # Standard errors
#print size of coefficients
length(coefficients)
length(t_values)
#df <- 5127
df<- gw.ms$GW.diagnostic$edf-gw.ms$GW.diagnostic$enp
# Calculate p-values assuming a large sample size (for simplicity, use normal approximation)
p_values_gw_ms <- 2 * (1 - pt(abs(t_values), df = df))

# Output coefficient, standard error, t-value, and p-value for 'min_distance_to_CAFs.myCAF.like'
result <- data.frame(
  Coefficient = coefficients,
  TValue = t_values,
  PValue = p_values_gw_ms
)


# filter result for p-values < 0.05
result_filt = result %>% filter(PValue<0.05)
#print shape of result_filt
dim(result_filt)
write.csv(result,"results/msgwr_tcell4_coef.csv")


df<- m.gwr$GW.diagnostic$edf-m.gwr$GW.diagnostic$enp

# do this for m.gwr
coefficients <- m.gwr$SDF$min_distance_to_T.cells.CD8.  # Coefficients
t_values <- m.gwr$SDF$min_distance_to_T.cells.CD8._TV # Standard errors
p_values <- 2 * (1 - pt(abs(t_values), df = df))
result <- data.frame(
  Location = 1:length(coefficients),
  Coefficient = coefficients,
  TValue = t_values,
  PValue = p_values
)
#save
#filter
result_filt = result %>% filter(PValue<0.05)
#print shape of result_filt
dim(result_filt)

write.csv(result,"results/gwr_tcell4_coef.csv")


[1] 5662

[1] 5662

[1] 2160    3

[1] 793   4

In [ ]:
variable_names <- names(gw.ms$lm$coefficients)
bandwidths <- gw.ms$GW.arguments$bws

# Create a data frame with variable names and corresponding bandwidths
bw_df <- data.frame(names = variable_names, bws = bandwidths)
#add m.gwr bw value m.gwr$GW.arguments$bw
bw_df = rbind(bw_df,data.frame(names="m.gwr",bws=m.gwr$GW.arguments$bw))
write.csv(bw_df,"results/bw_just_tme.csv")

In [ ]:
gw.ms

   ***********************************************************************
   *                       Package   GWmodel                             *
   ***********************************************************************
   Program starts at: 2024-11-02 00:28:08.265969 
   Call:
   gwr.multiscale(formula = formula, data = sf_data, kernel = "bisquare", 
    adaptive = T, criterion = "CVR", max.iterations = 1000, bws0 = bws0_list, 
    verbose = T, predictor.centered = rep(T, 4))

   Dependent (y) variable:  EMT_hallmarks
   Independent variables:  min_distance_to_Cancer.LumA.SC min_distance_to_Macrophage min_distance_to_Cancer.Basal.SC min_distance_to_Cancer.LumB.SC min_distance_to_Luminal.Progenitors min_distance_to_PVL.Immature min_distance_to_Mature.Luminal min_distance_to_Cancer.Her2.SC min_distance_to_Myoepithelial min_distance_to_CAFs.myCAF.like min_distance_to_Cancer.Cycling min_distance_to_Monocyte min_distance_to_Cycling_Myeloid min_distance_to_Endothelial.Lymphatic.LYVE1 m